<a href="https://colab.research.google.com/github/AaryaPakhale/Intern_Task3_Final/blob/main/Nexus_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163007 sha256=170a5433e9e04cba6f7c17fd74d40fab223cde65b3674e3e26a06069bc352cf7
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import SVD, KNNBasic
from surprise import accuracy

In [ ]:
user_ratings_df = pd.read_csv("ratings.csv",encoding='ISO-8859-1')
user_ratings_df.isna().sum()
user_ratings_df.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [ ]:
movie_metadata = pd.read_csv("movies_metadata.csv",encoding='ISO-8859-1', skiprows=lambda x: x == 5075)
movie_metadata =movie_metadata.rename(columns={'id': 'movieId'})
movie_names=movie_metadata[['title', 'genres']]
movie_metadata.head()

<ipython-input-3-99c6d429882c>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_metadata = pd.read_csv("movies_metadata.csv",encoding='ISO-8859-1', skiprows=lambda x: x == 5075)


,adult,belongs_to_collection,budget,genres,homepage,movieId,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
movie_metadata['movieId'] = movie_metadata['movieId'].astype(str)
user_ratings_df['movieId'] = user_ratings_df['movieId'].astype(str)
movie_data = user_ratings_df.merge(movie_metadata, on='movieId')

In [ ]:
user_rating_matrix=user_ratings_df.pivot(index=['userId'], columns=['movieId'], values='rating').fillna(0)
user_rating_matrix.head()

<ipython-input-5-3053eba41b06>:1: PerformanceWarning: The following operation may generate 3019639938 cells in the resulting pandas object.
  user_rating_matrix=user_ratings_df.pivot(index=['userId'], columns=['movieId'], values='rating').fillna(0)


In [ ]:
#filtering the movies based on ratings count
movie_counts = user_rating_matrix.sum(axis=0)
filtered_movies = movie_counts[movie_counts > 50].index
user_rating_matrix = user_rating_matrix.loc[:, filtered_movies]
user_rating_matrix.head()

In [ ]:
csr_data = csr_matrix(user_rating_matrix.values)
user_rating_matrix.reset_index(inplace=True)

In [ ]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(csr_data)

In [ ]:
!pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import process

def movie_recommender_engine(movie_name, matrix, cf_model, n_recs):
    # Fit model on matrix
    knn.fit(matrix)

    # Extract input movie ID
    movie_id = process.extractOne(movie_name, movie_names['title'])[2]

    # Calculate neighbour distances
    distances, indices = cf_model.kneighbors(matrix[movie_id], n_neighbors=n_recs)
    movie_rec_ids = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]

    # List to store recommendations
    cf_recs = []
    for i in movie_rec_ids:
        cf_recs.append({'Title':movie_names['title'][i[0]],'Distance':i[1]})

    # Select top number of recommendations needed
    df = pd.DataFrame(cf_recs, index = range(1,n_recs))

    return df

In [ ]:
n_recs = 14
movie_recommender_engine('Iron man', csr_data, knn, n_recs+1)


,Title,Distance
1,Escape from New York,0.725209
2,When a Man Loves a Woman,0.724886
3,Notorious,0.723352
4,Chinatown,0.719292
5,Raise the Red Lantern,0.716614
6,Koyaanisqatsi,0.716560
7,Gumby: The Movie,0.711370
8,Color of Night,0.710739
9,The Getaway,0.706452
10,Showgirls,0.706425


In [ ]:
#svd code for user-vise collaborative recommendations
from scipy.sparse.linalg import svds

def recommend_movies(user_rating_matrix, k, user_id, N=5):

  # Perform SVD with chosen k
  U, S, Vt = svds(user_rating_matrix, k)

  # Predict missing ratings
  predicted_ratings = np.dot(U, np.diag(S) @ Vt)

 # Recommend top N items
  user_ratings = predicted_ratings[user_id]
  item_indices = np.argsort(user_ratings)[::-1][:N]  # Sort and get top N indices
  return item_indices



In [ ]:
k = 3
user_rating_matrix = user_rating_matrix.to_numpy()
recommended_movies = recommend_movies(user_rating_matrix, k, user_id=1, N=3)
movie_list=[]
for i in recommended_movies:
  movie_list.append(movie_metadata.iloc[i]['title'])
print(f"Recommended movies for user 1: {movie_list}")


Recommended movies for user 1: ['Toy Story', 'Broken English', 'La Collectionneuse']


In [ ]:
#Code to find which model works better
data = pd.read_csv("ratings_small.csv",encoding='ISO-8859-1')
df = pd.DataFrame(data)
df = df.drop(columns = 'timestamp')

reader = Reader(rating_scale=(0,5))
data = Dataset.load_from_df(df , reader)

train_set , test_set = train_test_split(data ,test_size = 0.2 )
algorithms = [SVD(), KNNBasic()]

rmse_vals = []
mae_vals = []
for algo in algorithms:
  algo.fit(train_set)
  preds = algo.test(test_set)
  rmse = accuracy.rmse(preds)
  mae = accuracy.mae(preds)
  rmse_vals.append(rmse)
  mae_vals.append(mae)

combined_acc = [(rmse + mae) / 2 for rmse, mae in zip(rmse_vals, mae_vals)]

best_algo_index = combined_acc.index(min(combined_acc))
best_algo_name = algorithms[best_algo_index].__class__.__name__
print(f"Best Algorithm: {best_algo_name}")
print(f"RMSE: {rmse_vals[best_algo_index]}")
print(f"MAE: {mae_vals[best_algo_index]}")
print(f"Combined Score: {combined_acc[best_algo_index]}")